<hr>
<h2> 0. Global Constance & Variable</h2>
Config global values
<hr>

In [129]:
_DATASET_ADDRESS_ = './dataset/'
_PICKLE_ADDRESS_ = './pickle/'
_K_ = 6

_DELTA_ = 0.00005

In [178]:
#기본 라이브러리
import pandas as pd
import numpy as np
import math
import time
import pickle
import nashpy
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

tqdm.pandas()

# SVD 행렬축소
from sklearn.decomposition import TruncatedSVD

# 제작 함수
import func

In [131]:

#데이터 불러오기
ratings = pd.read_csv("u.data.txt", header=None, sep='\t')
ratings.columns = ['userid','movieid','rating','timestamp']
ratings = ratings.drop(['timestamp'], axis=1)
print("총 주어진 ratings 데이터 : ", ratings.shape)
rating_table= pd.pivot_table(ratings, values='rating', index=['userid'], columns=['movieid'])

총 주어진 ratings 데이터 :  (100000, 3)


In [132]:
# 80000 row
user_train_raw = pd.read_csv(_DATASET_ADDRESS_ + 'u1.base', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
user_train_raw = user_train_raw[['userID','itemID','rating']]
user_train = user_train_raw.pivot_table('rating', index = 'userID',columns = 'itemID').fillna(0)
user_train_raw = user_train_raw.pivot_table('rating', index = 'userID',columns = 'itemID')

# 20000 row
user_test = pd.read_csv(_DATASET_ADDRESS_ + 'u1.test', sep='\t',names=["userID","itemID","rating","timestamp"],header=None, na_filter=False)
user_test = user_test[['userID','itemID','rating']]
user_test = user_test.pivot_table('rating', index = 'userID',columns = 'itemID').fillna(0)

<h3>MF define</h3>

In [133]:
import numpy

def matrix_factorization(R, P, Q, K, steps=300, alpha=0.0002, beta=0.02):

    

    '''
    R: rating matrix
    P: |U| * K (User features matrix)
    Q: |D| * K (Item features matrix)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter
    '''
    Q = Q.T

    start = time.time()
    for step in range(steps):
        print('epoch: %d, time: %f'%(step, time.time()-start))
        for i in range(len(R)):
            for j in range(len(R[0])):
                if R[i][j] > 0:
                    # calculate error
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])

                    for k in range(K):
                        # calculate gradient with a and beta parameter
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])

        eR = numpy.dot(P,Q)

        e = 0

        for i in range(len(R)):

            for j in range(len(R[0])):

                if R[i][j] > 0:

                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)

                    for k in range(K):

                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        # 0.001: local minimum
        if e < 0.001:

            break

    return P, Q.T

In [134]:
# N: num of User
try:
    with open(_PICKLE_ADDRESS_ + 'user_train.mf.pkl', 'rb') as f: 
        user_train_mf = pickle.load(f)
        print("Found existing Result...")
except FileNotFoundError:
    print("No existing Result, Calculating...")

    user_train_np = user_train.to_numpy()

    N = len(user_train_np)
    # M: num of Movie
    M = len(user_train_np[0])

    K = 5  #이거를 [5, 10, 15, 20, 25, 30]
    
    P = np.random.rand(N,K)
    Q = np.random.rand(M,K)

    #   nP, nQ = matrix_factorization(user_train_np, P, Q, K)
    #user_train_mf = pd.DataFrame(nP@nQ.T)
    with open(_PICKLE_ADDRESS_ + 'user_train.mf.pkl', 'wb') as f:
        pickle.dump(user_train, f)

Found existing Result...


<h3>Train data :</h3>
<ol>
    <li>user_train_raw : rating with <strong>NaN</strong></li>
    <li>user_train     : rating filled with <strong>0</strong></li>
    <li>user_train_mf  : rating filled by <strong>MF</strong></li>
</ol>

In [135]:
#Test for loaded datas

user_train_mf.shape, user_train_raw.shape, user_train.shape

((943, 1650), (943, 1650), (943, 1650))

<br><br><br><br><br>
<br><br><br><br><br>
<hr>
<h2> 2. Group clustering with K-means</h2>
<hr>

In [136]:
from func import clustering

train_data, test_data, km_center = clustering(_K_, user_train_mf, user_train, user_test)

c:\_CSE\Python\Anaconda\envs\KDD\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


In [137]:
train_data.shape

(942, 1651)

<br><br><br><br><br>
<br><br><br><br><br>
<hr>
<h2> 3. Group recommendation</h2>
<ol>
    <li> 1) AVG</li>
    <li> 2) LM</li>
    <li> 3) Nash</li>
</ol>
<hr>

<h3>1) AVG</h3>

In [138]:
user_train_raw.shape
user_train_raw[1000][943]

nan

<h3>3) Nash</h3>

In [139]:
def getStat(rating) :
    ret = []
    for idx, item in rating.T.iterrows() :
        ret.append([item.count(), item.sum()])
    return ret

In [140]:
def deltaMatrix(rating) :
    ret = []
    for uidx, user in rating.iterrows() :
        iidx = 0
        tmp = []
        for item in user :
            userMean = stat[iidx][1] / stat[iidx][0]
            hasRated = not math.isnan(item)
            deltaSum = stat[iidx][1] - (item if hasRated else 0)
            deltaCount = stat[iidx][0] - (1 if hasRated else 0)
            tmp.append((userMean * deltaCount + _DELTA_) / (deltaSum + _DELTA_))
            iidx += 1
        ret.append(tmp)
    return np.array(ret)

In [141]:
for i in range(1,len(stat)) :
    10

In [151]:
def payoff(C, U, rating) :
    delta_train = deltaMatrix(rating)
    stat = getStat(rating)
    ret1 = []
    ret2 = []
    d = []
    for i in range(1,len(stat)) :
        hasCRated = not math.isnan(rating.iloc[C, i])
        hasURated = not math.isnan(rating.iloc[U, i])
        deltaSum = stat[i][0] - (rating.iloc[C, i] if hasCRated else 0) - (rating.iloc[U, i] if hasURated else 0)
        deltaCount = stat[i][1] - (1 if hasCRated else 0) - (1 if hasURated else 0)
        d.append((stat[i][1] / stat[i][0] * deltaCount + _DELTA_) / (deltaSum + _DELTA_))
    
    print(delta_train.shape)
    for i in range(1,len(stat)) :
        tmp = []
        for j in range(1,len(stat)) :
            tmp.append(delta_train[C][i] / (d[i-1] if i != j else delta_train[U][j]))
        ret1.append(tmp)
    for i in range(1,len(stat)) :
        tmp = []
        for j in range(1,len(stat)) :
            tmp.append(delta_train[U][i] / (d[i-1] if i != j else delta_train[C][j]))
        ret2.append(tmp)
    
    return np.array(ret1), np.array(ret2)


In [152]:
payoffMatrix = payoff(1, 2, user_train_raw)

(943, 1650)


In [183]:
payoffMatrix[0].shape, payoffMatrix[1].shape
list(payoffMatrix[1][1].shape)
plt.scatter()


[1649]

In [175]:
game = nashpy.Game(payoffMatrix[0], payoffMatrix[1])
result = game.lemke_howson_enumeration()

In [176]:
for i in result :
    if i != 0:
            print(i, " ")

(array([1., 0., 0., ..., 0., 0., 0.]), array([1., 0., 0., ..., 0., 0., 0.]))  
(array([0., 1., 0., ..., 0., 0., 0.]), array([0., 1., 0., ..., 0., 0., 0.]))  
(array([0., 0., 1., ..., 0., 0., 0.]), array([0., 0., 1., ..., 0., 0., 0.]))  
(array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]))  
(array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]))  
(array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]))  
(array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]))  
(array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]))  
(array([ 5.59947321e-05, -0.00000000e+00, -0.00000000e+00, ...,
        5.35246790e-04,  0.00000000e+00,  0.00000000e+00]), array([nan, nan, nan, ..., nan, nan, nan]))  
(array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]))  
(array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]))  
(array([0., 0., 0., ..., 0., 0., 0.]), a

KeyboardInterrupt: 